### How To Use The Packages - NeuralUCB and NeuralTS and LinUCB - A Comparision ###

In [ ]:
# # IMPORT STATEMENTS - Some Examples
# from bluebandits.NeuralUCB.agent import *
# from bluebandits.LinUCB.agent import *

# from bluebandits.NeuralThompsonSampling import *
# from bluesimulator.NeuralThompsonSampling.environment import *
# from bluesimulator.NeuralUCB.environment import *


In [1]:
# Import Dependencies
from tqdm import tqdm
import numpy as np
import bluebandits as bb


In [2]:
# Set the parameters
np.random.seed(12345)

# Set the parameter of the network
# the setting is based on the description of section 7.1 of the papaer
L = 2
m = 30 
K = 4# Total number of actions, 
T = 2666 # Total number of periods
d = 4 # the dimension of context
# we fix gamma in each round, according to the description of section 3.1
gamma_t = 0.01 #{0.01, 0.1, 1, 10}
nu = 0.1 #{0.01, 0.1, 1}
lambda_ = 1 #{0.1, 1, 10}
delta = 0.01 #{0.01, 0.1, 1}
S = 0.01 #{0.01, 0.1, 1, 10}
eta = 1e-3 #{0.001, 0.01, 0.1}
frequency = 50
batchsize = 50
verbose = False
A = np.random.normal(loc=0, scale=1, size=(d, d))
# Set the parameter of the network
theta = np.empty(shape = (T,K,d))
A1 = np.array([np.diag(np.ones(shape=d)) for _ in np.arange(K)])
p = np.empty(shape = (T, K))
X = np.array([[np.random.uniform(low=0, high=1, size=d) for _ in range(K)] for _ in np.arange(T)])
alpha = 1

In [ ]:
best_action = []
action_rank_UCB = []
action_rank_TS = []
action_rank_LinUCB = []
action_list_UCB = []
action_list_TS =[]
action_list_LinUCB = []

from bluebandits.NeuralUCB.agent import Agent
neuralUCBagent = Agent(
    K=K, T=T, d=d, L=L, m=m, gamma_t=gamma_t, nu=nu, lambda_=lambda_, delta=delta, S=S, eta=eta, frequency=frequency, batchsize=batchsize
)
from bluebandits.NeuralThompsonSampling.agent import Agent
neuralTSagent = Agent(K=K, T=T, d=d, L=L, m=m, nu=nu, lambda_=lambda_, eta=eta, frequency=frequency, batchsize=batchsize)
from bluebandits.LinUCB.agent import Agent
linearagent = bb.LinUCB.agent.Agent(K = K,T=T,d=d,A=A1,theta=theta,X=X,p=p,alpha=alpha)
for tt in tqdm(range(1, T + 1)):

    # if tt %2000 == 1:
    #     A = np.random.normal(loc=0, scale=1, size=(d, d))

    
    ''' God Mode'''
    context_list = SampleContext(d, K)
    realized_reward = GetRealReward(context_list, A)
    best_action_for_customer = np.argmax(realized_reward)
    best_action.append(best_action_for_customer)
    action_order = np.flip(np.argsort(realized_reward))

    ''' God Mode ends'''
    
    neuralUCB_ind = neuralUCBagent.Action(context_list)
    action_list_UCB.append(neuralUCB_ind)
    action_rank_UCB.append(list(action_order).index(neuralUCB_ind))

    neuralTS_ind = neuralTSagent.Action(context_list)
    action_list_TS.append(neuralTS_ind)
    action_rank_TS.append(list(action_order).index(neuralTS_ind))

    linearUCB_ind = linearagent.Action(context_list)
    action_list_LinUCB.append(linearUCB_ind)
    action_rank_LinUCB.append(list(action_order).index(linearUCB_ind))




    ''' Online Learning '''
    neuralUCB_reward = realized_reward[neuralUCB_ind]
    neuralUCBagent.Update(neuralUCB_reward)

    neuralTS_reward = realized_reward[neuralTS_ind]
    neuralTSagent.Update(neuralTS_reward)

    LinUCB_reward = realized_reward[linearUCB_ind]
    linearagent.Update(LinUCB_reward)

    